In [ ]:
import pandas as pd
import numpy as np

Importing the error filled data for each attribute (Fc=1,Tc=2,Tr=3, Tj=4) and concatenating for model training.

In [ ]:
data_no = pd.read_csv('FinalData/no_error.csv')
data_no["Error"]=0
dataFc = pd.read_csv('FinalData/Fc_data.csv')
dataTc = pd.read_csv('FinalData/Tc_error.csv')
dataTr = pd.read_csv('FinalData/Tr_error.csv')
dataTj = pd.read_csv('FinalData/Tj_error.csv')

data_no = data_no.iloc[:,2:]
dataTc = dataTc.iloc[:,2:]
dataTr = dataTr.iloc[:,2:]
dataTj = dataTj.iloc[:,2:]

data = pd.concat([data_no,dataFc,dataTc,dataTr,dataTj],ignore_index=True)
data['Error'].unique()


array([0, 1, 2, 3, 4], dtype=int64)

In [ ]:
data = data.drop(['Unnamed: 0', 'Current'],axis=1)

In [ ]:
data.count()

Fc       133900
Tc       133900
Tr       133900
Tj       133900
Error    133900
dtype: int64

Not using Undersampling since its time series data and dupicates in data will remaing making the model overfit

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(sampling_strategy='majority')
# X,y = rus.fit_resample(X,y)

removing all the duplicates from data

In [ ]:
data.drop_duplicates(inplace=True)

Spitting the data for the traing and testing

In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

Scaling the data

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train= sc.fit_transform(X_train)
X_test = sc.transform(X_test)

c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:767: FutureWarning: is_sparse is deprecated and wil

In [ ]:
from sklearn.svm import SVC
from sklearn import metrics as met

GridSearchCV, with parameter tuning

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters_dictionary = {
                         'kernel':['rbf'],
                         'C':[0.01,0.1,0.5,1,10,100],
                         'gamma':[0.01,0.1,1,10,100,1000,'scale'],
                         'decision_function_shape':['ovr','ovo'],
                         'degree':[1,2,3]
                         }
svc = SVC(class_weight={0:10,1:10,2:10,3:10,4:10})
grid_search = GridSearchCV(svc,
                           parameters_dictionary,
                           cv = 3,
                           scoring = 'accuracy',
                           verbose = 4,
                          )
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 252 candidates, totalling 756 fits


c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\ASUS\miniconda3\envs\mini-project-env\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


[CV 1/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.01, kernel=rbf;, score=0.421 total time= 3.4min
[CV 2/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.01, kernel=rbf;, score=0.426 total time= 3.5min
[CV 3/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.01, kernel=rbf;, score=0.433 total time= 3.3min
[CV 1/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.1, kernel=rbf;, score=0.574 total time= 2.5min
[CV 2/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.1, kernel=rbf;, score=0.581 total time= 2.2min
[CV 3/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=0.1, kernel=rbf;, score=0.582 total time=  57.3s
[CV 1/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=1, kernel=rbf;, score=0.771 total time=  36.6s
[CV 2/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=1, kernel=rbf;, score=0.769 total time=  37.0s
[CV 3/3] END C=0.01, decision_function_shape=ovr, degree=1, gamma=1, kernel=rbf;,

GridSearchCV(cv=3,
             estimator=SVC(class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}),
             param_grid={'C': [0.01, 0.1, 0.5, 1, 10, 100],
                         'decision_function_shape': ['ovr', 'ovo'],
                         'degree': [1, 2, 3],
                         'gamma': [0.01, 0.1, 1, 10, 100, 1000, 'scale'],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=4)

In [ ]:
grid_search.best_estimator_

SVC(C=0.5, class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, degree=1,
    gamma=100)

In [ ]:
grid_search.best_params_

{'C': 0.5,
 'decision_function_shape': 'ovr',
 'degree': 1,
 'gamma': 100,
 'kernel': 'rbf'}

In [ ]:
grid_search.best_score_*100

98.33440298279393

In [ ]:
grid_search.cv_results_['mean_test_score']

array([0.42672901, 0.57883317, 0.76858898, 0.93169524, 0.9728309 ,
       0.83875798, 0.67763821, 0.42672901, 0.57883317, 0.76858898,
       0.93169524, 0.9728309 , 0.83875798, 0.67763821, 0.42672901,
       0.57883317, 0.76858898, 0.93169524, 0.9728309 , 0.83875798,
       0.67763821, 0.42672901, 0.57883317, 0.76858898, 0.93169524,
       0.9728309 , 0.83875798, 0.67763821, 0.42672901, 0.57883317,
       0.76858898, 0.93169524, 0.9728309 , 0.83875798, 0.67763821,
       0.42672901, 0.57883317, 0.76858898, 0.93169524, 0.9728309 ,
       0.83875798, 0.67763821, 0.47706366, 0.74527062, 0.87463708,
       0.9650683 , 0.98202989, 0.96818557, 0.77335656, 0.47706366,
       0.74527062, 0.87463708, 0.9650683 , 0.98202989, 0.96818557,
       0.77335656, 0.47706366, 0.74527062, 0.87463708, 0.9650683 ,
       0.98202989, 0.96818557, 0.77335656, 0.47706366, 0.74527062,
       0.87463708, 0.9650683 , 0.98202989, 0.96818557, 0.77335656,
       0.47706366, 0.74527062, 0.87463708, 0.9650683 , 0.98202

In [ ]:
grid_search.best_index_

88

Creating the dataframe for each combination and sorted based on accuracy.

In [ ]:
results = grid_search.cv_results_

C_values = [params['C'] for params in results['params']]
kernel_values = [params['kernel'] for params in results['params']]
gamma_values = [params['gamma'] for params in results['params']]
dec_values = [params['decision_function_shape'] for params in results['params']]
scores = results['mean_test_score']

data = {
    'C': C_values,
    'Kernel': kernel_values,
    'gamma': gamma_values,
    'Decsion-Function':dec_values,
    'Score': scores*100,
}

parameterTuningData = pd.DataFrame(data)
sortedData = parameterTuningData.sort_values(by=['Score'],ascending=False)

sortedData.to_csv('FinalDataBestFit_DataFrame/Newdata_par.csv',index=False)
sortedData

,C,Kernel,gamma,Decsion-Function,Score
116,0.50,rbf,100,ovo,98.334403
109,0.50,rbf,100,ovo,98.334403
123,0.50,rbf,100,ovo,98.334403
95,0.50,rbf,100,ovr,98.334403
88,0.50,rbf,100,ovr,98.334403
...,...,...,...,...,...
28,0.01,rbf,0.01,ovo,42.672901
21,0.01,rbf,0.01,ovo,42.672901
14,0.01,rbf,0.01,ovr,42.672901
7,0.01,rbf,0.01,ovr,42.672901


Creating the Pickle File for each model for finla training the model.

In [ ]:
import joblib
from sklearn.model_selection import ParameterGrid
from shutil import copyfile

parameters_dictionary = {
                         'kernel':['rbf'],
                         'C':[0.1,0.5,1,10,100],
                         'gamma':[1,10,100,1000,'scale'],
                         'decision_function_shape':['ovo','ovr'],
                        #  'degree':[1,2,3]
                         }
i = 0
for params in ParameterGrid(parameters_dictionary):
    svc = SVC(class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, **params)
    filename = f'F_Model_{i}_Kernel_{params["kernel"]}_C_{params["C"]}_gamma_{params["gamma"]}_decision_{params["decision_function_shape"]}.pkl'
    svc.fit(X_train, y_train)
    joblib.dump(svc, filename)
    source_path = filename
    target_path = 'FinalDataNewModelPickleFile_z/'+f'{filename}'
    copyfile(source_path, target_path)
    print(f"Model saved for combination: {params}")
    i = i+1

Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 1, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 10, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 100, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 1000, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovo', 'gamma': 'scale', 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovr', 'gamma': 1, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovr', 'gamma': 10, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovr', 'gamma': 100, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_function_shape': 'ovr', 'gamma': 1000, 'kernel': 'rbf'}
Model saved for combination: {'C': 0.1, 'decision_funct

Below is the orginal paramter grid that has to be evaluated through the GridSearchCV

In [ ]:
parameters_dictionary = {
                         'kernel':['rbf', 'poly', 'linear','sigmoid'],
                         'C':[0.1,0.2,0.5,1,10,100],
                         'gamma':[0.1,1,10,100,1000,'scale','auto'],
                         'decision_function_shape':['ovo','ovr'],
                         'degree':[2,3]
                         }

In [ ]:
import glob
import joblib
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.metrics import cohen_kappa_score, matthews_corrcoef, average_precision_score, log_loss, balanced_accuracy_score

# Step 1: Reading Pickle Files
pickle_files = glob.glob('FinalDataNewModelPickleFile_z/*.pkl')  # Replace 'path_to_folder' with the actual folder path
models_performance = []

for file in pickle_files:
    model = joblib.load(file)

    # Step 2: Verifying Classification Performance
    y_pred = model.predict(X_test)  # Assuming X_test is your test data
    accuracy = accuracy_score(y_test, y_pred)
    precision_w = precision_score(y_test, y_pred, average='weighted')
    recall_w = recall_score(y_test, y_pred, average='weighted')
    f1_w = f1_score(y_test, y_pred, average='weighted')
    precision_m = precision_score(y_test, y_pred, average='macro')
    recall_m = recall_score(y_test, y_pred, average='macro')
    f1_m = f1_score(y_test, y_pred, average='macro')
    cohen_kappa = cohen_kappa_score(y_test, y_pred)
    matthews_corr = matthews_corrcoef(y_test, y_pred)
    # avg_precision = average_precision_score(y_test, y_pred)
    try:
        logloss = log_loss(y_test, model.predict_proba(X_test))
    except:
        logloss = "NA"
    balanced_acc = balanced_accuracy_score(y_test, y_pred)

    # Step 3: Saving Results in a DataFrame
    model_performance = {
        'Model': file,
        'Accuracy': accuracy,
        'Precision_w': precision_w,
        'Recall_w': recall_w,
        'F1-Score_w': f1_w,
        'Precision_m': precision_m,
        'Recall_m': recall_m,
        'F1-Score_m': f1_m,
        'Cohen\'s Kappa': cohen_kappa,
        'Matthews Corr. Coef.': matthews_corr,
        'Log Loss': logloss,
        'Balanced Accuracy': balanced_acc
    }
    models_performance.append(model_performance)

# Create a dataframe from the list of model performances
performance_df = pd.DataFrame(models_performance)

In [ ]:
performance_df.to_csv("FinalDataNewModelPickleFile_z/FFP_Model_outputFinal.csv")